<a href="https://colab.research.google.com/github/Mamtakurrey123/Pytorch-Tutorial/blob/main/GAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch-geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.1 MB/s eta 0:00:00


In [3]:
import torch
import torch.nn as nn
from torch_geometric.nn import GATConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import numpy as np
import os


In [4]:
# Define GAT-based congestion prediction model
class GATCongestionPredictor(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_heads):
        super(GATCongestionPredictor, self).__init__()
        self.gat1 = GATConv(input_dim, hidden_dim, heads=num_heads, concat=True)
        self.gat2 = GATConv(hidden_dim * num_heads, output_dim, heads=1, concat=False)

    def forward(self, x, edge_index):
        x = self.gat1(x, edge_index)
        x = torch.relu(x)
        x = self.gat2(x, edge_index)
        return torch.sigmoid(x)

In [5]:
# Load real GCELL features and edge list
def load_routing_grid(feature_file="gcell_features.npy", edge_file="edge_index.npy"):
    """Load real GCELL features and edges from files."""
    if not os.path.exists(feature_file) or not os.path.exists(edge_file):
        raise FileNotFoundError("Feature or edge file not found. Please prepare gcell_features.npy and edge_index.npy.")

    node_features = np.load(feature_file)  # [N_nodes, features]
    edge_index = np.load(edge_file)        # [2, num_edges]

    x = torch.tensor(node_features, dtype=torch.float)
    edge_index = torch.tensor(edge_index, dtype=torch.long)
    return x, edge_index


In [16]:
# Train the GAT model
def train_gat_model(feature_file="gcell_features.npy", edge_file="edge_index.npy"):
    x, edge_index = load_routing_grid(feature_file, edge_file)
    # Generate synthetic congestion labels for training (random)
    congestion_labels = torch.rand((x.shape[0], 1))  # You can replace this with real congestion labels if available
    # Further training logic can be added here
    # ...
    # Create the dataset #This line and the next line were previously indented too far
    data = Data(x=x, edge_index=edge_index, y=congestion_labels)
    dataloader = DataLoader([data], batch_size=1, shuffle=True)
    # Initialize the GAT model #These next 3 lines were previously not indented enough
    model = GATCongestionPredictor(input_dim=x.shape[1], hidden_dim=8, output_dim=1, num_heads=2)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()
    # Training loop
    model.train()
    for epoch in range(100):
        for batch in dataloader:
            optimizer.zero_grad()
            output = model(batch.x, batch.edge_index)
            loss = loss_fn(output, batch.y)
            loss.backward()
            optimizer.step()
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}, Loss: {loss.item():.6f}")

    print("Training completed.")
    return model, data

In [18]:
# Save GAT predictions (no need for coords)
def save_gat_predictions(model, data, output_path="gat_congestion_predictions.npy"):
    model.eval()
    with torch.no_grad():
        predictions = model(data.x, data.edge_index).squeeze().cpu().numpy()
    np.save(output_path, predictions)
    print(f"GAT congestion predictions saved to {output_path}")

if __name__ == "__main__":
    # Set your feature and edge files
    feature_file = "data/gcell_features.npy"
    edge_file = "data/edge_index.npy"

    # Train or load model
    model, data = train_gat_model(feature_file, edge_file)
    torch.save(model.state_dict(), "gat_congestion_model.pt")

    # Save predictions (directly save array)
    save_gat_predictions(model, data)


FileNotFoundError: Feature or edge file not found. Please prepare gcell_features.npy and edge_index.npy.